# Multisignature and P2SH (pay to script hash) scripts in Bitcoin

## 1. Multisignature script

In [3]:
import bitcoin  #pybitcointools
from itertools import product
import binascii

In [4]:
#Initializing

M = 2 #Minimum number of signatures to approve the transaction (quorum)
N = 3 #Number of all the signatures that can be provided
priv_keys = [bitcoin.random_key() for _ in range(N)]
pub_keys = [bitcoin.privkey_to_pubkey(priv_key) for priv_key in priv_keys]
tx = 'Transaction example'
tx_hash = bitcoin.txhash(tx)

In [5]:
#Locking script building

locking_script = '{M} {pub_keys[0]} {pub_keys[1]} {pub_keys[2]} {N} CHECKMULTISIG'.format(M=M,
                                                                                          pub_keys=pub_keys,
                                                                                          N=N)
print('Multisig locking script:', locking_script, sep='\n\n')

Multisig locking script:

2 04a7ee4a9d87975b45133a5055d3a49efd5e021f888075c43798c34595990bdf0321af3e79cf188263be8a122870c4f1064cf2e4dbf9da2cbbe3d1693a5b8a47fc 04050e9f2915eeb069599eb1b9eb340b18637ab1f02cc5a9d9ef50c6a3abe66b741eee4882a28128ee2b59c345a2c526e1bcd9b507954520eb70e4f3aade3b2c15 04de2a82e3204697bf7ab35d74854fb683f8737c1532547aa41071933f78a0636a2ddb6eb4e1763b6788e6ce244155d6718c5b9f503244f35f6a667f9796348523 3 CHECKMULTISIG


In [6]:
#Making signatures (i.e. unlocking scripts)

sigs = [bitcoin.ecdsa_raw_sign(tx_hash, priv_key) for priv_key in priv_keys]
sigs
der_encoded_sigs = [bitcoin.der_encode_sig(*sig) for sig in sigs]
for i in range(N):
    print('Signature {i}: {sig}'.format(i=i, sig=der_encoded_sigs[i]), end='\n\n')

Signature 0: 304402202b26ca18e05fd7493f3bbdb34ae13b9594752e6276d75505021f289db8956295022011431d929288ecef4ebb01c6cb69502623d987f43248c3591222f067aada9430

Signature 1: 3045022100b2c8f855b1d5e53483219939bbfd48670c3d5e5ad973f5ccd0c244d94d76877a02206b5e53d05d6b6bdcb7a8b6b8ffb38f2d1f04b353c541e4d40d87aa73c8a70ad4

Signature 2: 3044022015c0bd2c9252a589f811ce6b4359199d3918cd6d84fa8a48e7aa68f91907603802203e8541c3c67a7d3d203312070922e6a5b6d7258877cd8e946bf5fa6c8b9d2c24



In [7]:
#Unlocking script building
#Starting with 0 by convention

unlocking_script = '0 {sigs[1]} {sigs[2]}'.format(sigs=der_encoded_sigs)
print('Multisig unlocking script:', unlocking_script, sep='\n')

Multisig unlocking script:
0 3045022100b2c8f855b1d5e53483219939bbfd48670c3d5e5ad973f5ccd0c244d94d76877a02206b5e53d05d6b6bdcb7a8b6b8ffb38f2d1f04b353c541e4d40d87aa73c8a70ad4 3044022015c0bd2c9252a589f811ce6b4359199d3918cd6d84fa8a48e7aa68f91907603802203e8541c3c67a7d3d203312070922e6a5b6d7258877cd8e946bf5fa6c8b9d2c24


In [8]:
#Pushing both scripts to a stack
#Comparing each signature with each public key until reaching M successful verifications

def multisig_verify(lock, unlock, tx_hash):
    stack = unlock.split() + lock.split()
    if stack.pop() == 'CHECKMULTISIG':
        N = int(stack.pop())
        pub_keys = []
        sigs = []
        for _ in range(N):
            pub_keys.append(stack.pop())
        M = int(stack.pop())
        for _ in range(M+1):
            sigs.append(stack.pop())
        sigs = [bitcoin.der_decode_sig(sig) for sig in sigs]
        n_verified = 0
        multi_verified = False
        for sig, pub_key in product(sigs, pub_keys):
            if bitcoin.ecdsa_raw_verify(tx_hash, sig, pub_key):
                n_verified += 1
            if n_verified == M:
                multi_verified = True
                break
        return multi_verified
    
if multisig_verify(locking_script, unlocking_script, tx_hash):
    print('Successfully verified!')

Successfully verified!


## 2. P2SH script

In [9]:
#Constructing P2SH script

#Arranging data types
splitted = locking_script.split()
splitted[0] = int(splitted[0]) 
splitted[4] = int(splitted[4])
splitted[5] = 174  #CHECKMULTSIG opcode

#Serializing and hashing
serialized = bitcoin.serialize_script(splitted)
redeem_hash = bitcoin.hash160(binascii.unhexlify(serialized))
p2sh_locking = 'HASH160 {} EQUAl'.format(redeem_hash)
print('P2SH script:', p2sh_locking, sep='\n\n')

P2SH script:

HASH160 975b081d08842448dee89908e9161d55e81d2815 EQUAl


In [10]:
#P2SH address (version prefix 5)
p2sh_address = bitcoin.hex_to_b58check(redeem_hash, magicbyte=5)
print('P2SH address:', p2sh_address, sep='\n\n')

P2SH address:

3FVK1W8Z973c1ucTfXqRaeqgFbK9ny9Kd2
